In [10]:
import random
import re

retriever = db.as_retriever(search_kwargs={'k': 50})

# 저장된 출력 공고 목록을 추적할 변수
output_history = []
user_input_data = {}  # 사용자 입력을 저장할 변수

def get_user_input():
    job = input('원하시는 직무를 입력하세요: ')
    exp = input('신입 / 경력 중 해당하는 내용을 입력하세요: ')
    loc = input('원하시는 지역을 입력하세요: ')
    return job, exp, loc

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain, ConversationChain

system_message = """
    너는 입력된 quesetion 속에 포함된 직무, 경력, 선호 지역에 맞는 채용 공고를 다섯 개씩 출력하는 탐색 AI야.
    검색된 context들 중 입력된 직무와 같은 카테고리인 채용 공고를 찾아오면 돼.
    공고이름에 직무가 들어가는 공고를 우선으로 출력하고, 그 이외에는 랜덤으로 출력해줘.
    만약 사용자가 더 많은 공고를 필요로 한다면, 같은 카테고리에서 이전에 네가 가져온 공고들을 제외하고 나머지 공고들 중 다섯 개를 뽑아서 가져와서 출력해줘.
    출력 형식은 하나의 공고마다 회사이름, 공고이름, url를 출력해주면 돼.

    # context: {context}
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{question}'),
])
llm = ChatGoogleGenerativeAI(
    model = 'gemini-1.5-flash',
    temperature=0.1
)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

def load_memory(input):
    return memory.load_memory_variables({})['chat_history']

def extract_company_and_title(page_content):
    company_pattern = r"회사이름: ([^,]+)"
    job_title_pattern = r"공고이름: ([^,]+)"
    url_pattern = r"url: ([^ ]+)"
    
    company_name = re.search(company_pattern, page_content)
    job_title = re.search(job_title_pattern, page_content)
    url = re.search(url_pattern, page_content)
    
    return company_name.group(1) if company_name else None, job_title.group(1) if job_title else None, url.group(1) if url else None

def prioritize_job_title(results, job):
    job_related = []
    non_job_related = []
    
    for result in results:
        company_name, job_title, url = extract_company_and_title(result.page_content)
        
        if job_title and job.lower() in job_title.lower():
            job_related.append(result)
        else:
            non_job_related.append(result)
    
    return job_related + random.sample(non_job_related, min(5, len(non_job_related)))

def chatbot():
    global user_input_data  # 사용자 입력 데이터를 전역 변수로 사용
    if not user_input_data:
        # 사용자로부터 직무, 경력, 지역 입력 받기 (처음 한 번만)
        job, exp, loc = get_user_input()
        user_input_data = {"job": job, "exp": exp, "loc": loc}
    else:
        # 이미 입력된 데이터 사용
        job = user_input_data["job"]
        exp = user_input_data["exp"]
        loc = user_input_data["loc"]
    
    question = f'{loc}에서 {exp}을 채용하는 {job} 공고 알려줘'
    
    # retriever에서 더 많은 결과를 가져오도록 설정 (예: 10개)
    results = retriever.get_relevant_documents(question, limit=10)
    
    # 직무가 제목에 포함된 공고를 우선적으로 가져오기
    results = prioritize_job_title(results, job)
    
    global output_history
    if output_history:
        # 이전에 출력된 결과는 제외하고 새로운 공고만 출력
        results = [result for result in results if result not in output_history]
    
    output = results[:5]  # 새로운 공고 중에서 5개 출력
    output_history.extend(output)  # 출력된 공고를 기록
    
    # 결과 출력
    for result in output:
        company_name, job_title, url = extract_company_and_title(result.page_content)
        print(f"회사이름: {company_name}, 공고이름: {job_title}, URL: {url}")
    
    memory.save_context({'input': question}, {'output': [f"회사이름: {company_name}, 공고이름: {job_title}, URL: {url}" for result in output]})
    
    # 더보기 여부 처리
    more_query = input("더 보기를 원하시면 'yes'를 입력하세요.")
    if more_query.lower() == 'yes':
        chatbot()  # 사용자 입력 없이 더보기 결과를 출력

# 챗봇 실행
chatbot()

c:\Users\user\anaconda3\envs\langchain1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\user\AppData\Local\Temp\ipykernel_36660\626067911.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
C:\Users\user\AppData\Local\Temp\ipykernel_36660\626067911.py:87: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(question, limit=10)


회사이름: (주)퍼플아카데미, 공고이름: 데이터 분석가 신입/경력 채용, URL: https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=49475791&location=ts&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D%EA%B0%80&searchType=search&paid_fl=n&search_uuid=e2706bf0-b6fc-4323-89b6-6f8223f7c2b0,
회사이름: (주)데이터누리, 공고이름: [데이터누리] 데이터 분석가를 모십니다., URL: https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=49257728&location=ts&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D%EA%B0%80&searchType=search&paid_fl=n&search_uuid=e2706bf0-b6fc-4323-89b6-6f8223f7c2b0,
회사이름: 비웨이브(주), 공고이름: 데이터 분석가 채용 공고, URL: https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=49447291&location=ts&searchword=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D%EA%B0%80&searchType=search&paid_fl=n&search_uuid=2815e49e-6ff3-4892-93a6-b488abe61dc2,
회사이름: (주)엠포스, 공고이름: [엠포스] 데이터 분석가 경력직 채용, URL: https://www.saramin.co.kr/zf_user/jobs/relay/view?view_type=search&rec_idx=49539376&location

In [2]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

In [4]:
# jobs.txt 파일에서 채용공고 데이터 읽기
with open('../jobs.txt') as f:
    file = f.read()


In [5]:
file = file.replace('\n', '\n\n')

In [6]:
from langchain_text_splitters import CharacterTextSplitter
# 텍스트를 분할하기 위한 설정: tiktoken 기반 텍스트 분할기 생성
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500  # 각 청크의 최대 크기
)

# 텍스트를 설정된 크기로 분할하여 리스트 생성
lines = splitter.split_text(file)

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_texts(texts=lines, embedding=embeddings)

In [9]:
retriever = db.as_retriever()

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.schema import HumanMessage
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 1. 채용 공고 데이터 검색 준비 (FAISS 데이터베이스 로드)
from langchain.embeddings import OpenAIEmbeddings

# 임베딩 생성
embeddings = OpenAIEmbeddings()

# FAISS 데이터베이스 로드
# 데이터베이스를 생성하지 않은 경우 jobs.txt 데이터로 생성하세요.
try:
    db = FAISS.load_local(
    "faiss_jobs_db",
    embeddings,
    allow_dangerous_deserialization=True
)
except FileNotFoundError:
    print("FAISS 데이터베이스가 없습니다. 'jobs.txt' 파일로 데이터베이스를 생성하세요.")
    exit()

retriever = db.as_retriever()

# 2. LLM 모델 설정
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1
)

# 3. Prompt Template 작성
prompt_template = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template="""
    다음은 대화 기록입니다:
    {chat_history}

    다음은 채용 공고 데이터입니다:
    {context}

    질문: {question}
    답변:
    """
)

# 4. Memory 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 5. 챗봇 구성
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt_template,
    input_key="question",
    output_key="answer"
)

# 6. 사용자 입력 및 챗봇 응답
print("채용공고 챗봇에 오신 것을 환영합니다. 질문을 입력하세요. '종료'를 입력하면 종료됩니다.")

while True:
    user_input = input("질문: ")
    if user_input.strip().lower() == "종료":
        print("챗봇을 종료합니다.")
        break

    # 관련 채용 공고 검색
    related_docs = retriever.get_relevant_documents(user_input)
    context = "\n".join([doc.page_content for doc in related_docs])

    # 답변 생성
    response = conversation.predict(question=user_input, context=context)
    print(f"답변: {response}")

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open faiss_jobs_db\index.faiss for reading: No such file or directory